In [1]:
import requests
import time
import json
import os
import re
from datetime import datetime

In [2]:
folder_path = "data/"

# Regular expression to match the file format
pattern = re.compile(r"nextbike_data_(\d{8}_\d{6})")

files_with_dates = []

for filename in os.listdir(folder_path):
    match = pattern.match(filename)
    if match:
        date_str = match.group(1)
        try:
            file_datetime = datetime.strptime(date_str, "%Y%m%d_%H%M%S")
            full_path = os.path.join(folder_path, filename)
            files_with_dates.append((file_datetime, full_path))
        except ValueError:
            print(f"Skipping file with invalid datetime: {filename}")

files_with_dates.sort()
sorted_paths = [path for _, path in files_with_dates]

In [3]:
def extract_station_data(data):
    stations = {}
    city  = data["countries"][0]["cities"][0]
    for place in city.get("places", []):
        stations[place["uid"]] = {
            "name": place["name"],
            "bikes": set(bike["number"] for bike in place.get("bike_list", []))
        }
    return stations

In [4]:
def load_state(path):
    with open(path, "r") as f:
        state = json.load(f)
        return state

In [11]:
bike_movements = []
previous_state = extract_station_data(load_state(sorted_paths[0]))
for path in sorted_paths[1:5000]:
    try:
        current_state = extract_station_data(load_state(path))
        
        for station_id, station_info in previous_state.items():
            if station_id in current_state:
                disappeared_bikes = station_info["bikes"] - current_state[station_id]["bikes"]
                for bike in disappeared_bikes:
                    for new_station_id, new_station_info in current_state.items():
                        if bike in new_station_info["bikes"]:
                            movement = {
                                "bike_id": bike,
                                "from_station": station_info["name"],
                                "to_station": new_station_info["name"],
                                "timestamp": path  # assuming 'path' includes or represents a timestamp
                            }
                            bike_movements.append(movement)
                            print(f"Bike {bike} moved from {station_info['name']} to {new_station_info['name']}")

        previous_state = current_state
        print("-" * 20)
    except:
        pass

--------------------
Bike 18194 moved from virtuell - U Rathaus Neukölln (Boddinstraße) to BIKE 18194
Bike 13531 moved from Volksbühne (U Rosa-Luxemburg-Platz) | BONUS-Station: Return(Rückgabe) here=15 mins free to BIKE 13531
--------------------
--------------------
--------------------
--------------------
--------------------
Bike 15976 moved from virtuell - Möckernstraße/Hornstraße to Zeughofstraße (Lausitzer Park) | BONUS-Station: Return(Rückgabe) here=15 mins free
--------------------
--------------------
Bike 100159 moved from virtuell - U Kleistpark (Süd) to virtuell - S+U Innsbrucker Platz (Nord)
--------------------
Bike 17829 moved from virtuell - U Hansaplatz to BIKE 17829
Bike 17800 moved from virtuell - Wrangelstraße/Cuvrystraße to virtuell - Adelbertstraße/Melchiorstraße
--------------------
Bike 15404 moved from Jelbi Hasenheide/Graefestraße (KRE/HA) to BIKE 15404
--------------------
Bike 10689 moved from virtuell - Friedrichsstraße/Behrensstraße/Ost (Abstellfläche-Mik

In [ ]:
from collections import defaultdict

bike_movement_counts = defaultdict(int)

for movement in bike_movements:
    bike_id = movement["bike_id"]
    bike_movement_counts[bike_id] += 1

bike_movement_summary = sorted(bike_movement_counts.items(), key=lambda x: x[1], reverse=True)

for bike_id, count in bike_movement_summary:
    print(f"Bike {bike_id} moved {count} times")


Bike 17088 moved 20 times
Bike 19156 moved 18 times
Bike 17819 moved 18 times
Bike 100809 moved 18 times
Bike 15615 moved 17 times
Bike 13538 moved 16 times
Bike 19746 moved 16 times
Bike 18137 moved 16 times
Bike 19336 moved 15 times
Bike 18039 moved 14 times
Bike 100427 moved 14 times
Bike 10935 moved 14 times
Bike 100643 moved 14 times
Bike 100702 moved 13 times
Bike 13167 moved 12 times
Bike 100386 moved 12 times
Bike 15136 moved 12 times
Bike 17646 moved 11 times
Bike 17937 moved 10 times
Bike 15544 moved 10 times
Bike 16116 moved 10 times
Bike 14237 moved 10 times
Bike 13284 moved 10 times
Bike 14133 moved 9 times
Bike 14385 moved 9 times
Bike 19497 moved 8 times
Bike 18964 moved 8 times
Bike 16769 moved 8 times
Bike 14674 moved 8 times
Bike 17300 moved 7 times
Bike 14539 moved 7 times
Bike 17886 moved 7 times
Bike 17809 moved 7 times
Bike 15557 moved 7 times
Bike 13658 moved 7 times
Bike 13119 moved 7 times
Bike 19517 moved 6 times
Bike 16389 moved 6 times
Bike 17496 moved 6 tim

In [ ]:
target_bike_id = "19156"

target_bike_movements = [
    move for move in bike_movements if move["bike_id"] == target_bike_id
]

for move in target_bike_movements:
    print(f"{move['timestamp']}: Bike: {move['bike_id']} moved from {move['from_station']} to {move['to_station']}")


data/nextbike_data_20250403_071304.json: Bike: 19156 moved from U Weinmeisterstraße/Gormannstraße | BONUS-Station: Return(Rückgabe) here=15 mins free to BIKE 19156
data/nextbike_data_20250403_072117.json: Bike: 19156 moved from Jelbi U Weinmeisterstraße (MIT/WE) to Jelbi Linienstraße/Rückerstraße (MIT/RU)
data/nextbike_data_20250403_072625.json: Bike: 19156 moved from Jelbi Linienstraße/Rückerstraße (MIT/RU) to Jelbi Kleine Rosenthaler Straße/Steinstraße (MIT/RO)
data/nextbike_data_20250403_072727.json: Bike: 19156 moved from Jelbi Kleine Rosenthaler Straße/Steinstraße (MIT/RO) to Jelbi Neue Schönhauser Straße/Rosenthaler Straße (MIT/SR)
data/nextbike_data_20250403_072828.json: Bike: 19156 moved from Jelbi Neue Schönhauser Straße/Rosenthaler Straße (MIT/SR) to BIKE 19156
data/nextbike_data_20250403_073133.json: Bike: 19156 moved from M Memhardstraße | BONUS-Station: Return(Rückgabe) here=15 mins free to BIKE 19156
data/nextbike_data_20250403_073540.json: Bike: 19156 moved from virtuell